In [1]:
import os
import sys
import numpy as np
import keras
# from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
seed = 273
kinase = 'tpk_src'
model_file = 'rnn_mlp_early_%s.h5' % (kinase)

In [3]:
#import data
smiles_lines = [line.strip().split(',') for line in open(os.path.join('data', kinase+'_smiles.csv'))]
smiles = [line[1] for line in smiles_lines]
fingerprint_lines = [line.strip().split(',') for line in open(os.path.join('data', kinase+'_fingerprints.csv'))]
fingerprints = [line[2:] for line in fingerprint_lines]
X_fingerprints = np.asarray(fingerprints, dtype=np.int16)
y = np.asarray([int(line[2]) for line in smiles_lines], dtype=np.int8)

In [4]:
#pad smiles with '!' to ensure equal length
max_smiles_len = max(len(s) for s in smiles)
smiles = [s + '!'*(max_smiles_len + 1 - len(s)) for s in smiles]

In [5]:
#one-hot vector representation of smiles
char_set = set()
for s in smiles:
    for c in s:
        char_set.add(c)
char_set = list(char_set)
char_to_index = {char_set[i]: i for i in range(len(char_set))}

X_smiles = np.zeros((len(smiles), max_smiles_len + 1, len(char_set)))
for i in range(len(smiles)):
    code = smiles[i]
    for j in range(len(code)):
        char = code[j]
        X_smiles[i, j, char_to_index[char]] = 1        

In [6]:
print(X_fingerprints.shape)
print(X_smiles.shape)
print(y.shape)

(2972, 6117)
(2972, 416, 40)
(2972,)


In [7]:
#train val test split
n_tot = X_smiles.shape[0]
n_test = round(n_tot*0.15)
n_val = round(n_tot*0.15)
n_train  = n_tot - n_test - n_val
indices = np.arange(n_tot, dtype=int)
np.random.seed(seed)
np.random.shuffle(indices)
indices_train = indices[:n_train]
indices_val = indices[n_train:n_train+n_val]
indices_test = indices[n_train+n_val:]
X_fingerprints_train = X_fingerprints[indices_train]
print(X_fingerprints_train.shape)
X_smiles_train = X_smiles[indices_train]
print(X_smiles_train.shape)
y_train = y[indices_train]
print(y_train.shape)
X_fingerprints_val = X_fingerprints[indices_val]
print(X_fingerprints_val.shape)
X_smiles_val = X_smiles[indices_val]
print(X_smiles_val.shape)
y_val = y[indices_val]
print(y_val.shape)
X_fingerprints_test = X_fingerprints[indices_test]
print(X_fingerprints_test.shape)
X_smiles_test = X_smiles[indices_test]
print(X_smiles_test.shape)
y_test = y[indices_test]
print(y_test.shape)
smiles_input_shape = X_smiles_train.shape[1:]
fingerprints_input_shape = X_fingerprints_train.shape[1:]

(2080, 6117)
(2080, 416, 40)
(2080,)
(446, 6117)
(446, 416, 40)
(446,)
(446, 6117)
(446, 416, 40)
(446,)


In [8]:
#check GPU presence
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

[]


In [9]:
#keras imports
from keras.models import Sequential
from keras.layers import TimeDistributed, GlobalAveragePooling2D, Activation, Dense, Input, Bidirectional
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback, EarlyStopping
from keras.layers import Dropout, Flatten
from keras.layers import concatenate
from keras import regularizers
from keras import initializers
from keras import constraints
from keras.models import Model
# Backend
from keras import backend as K
# Utils
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import metrics

In [10]:
#F1 score computation
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [11]:
#basic MLP model
def rnn_mlp_early():
    si = Input(shape=smiles_input_shape)
    s = Bidirectional(LSTM(units=128, return_sequences=False))(si)
    fi = Input(shape=fingerprints_input_shape)
    o = concatenate([s,fi],axis=-1)
    o = Dense(128, activation='relu')(o)
    o = Dense(128, activation='relu')(o)
    o = Dropout(0.5)(o)
    out = Dense(2, activation='softmax')(o)
    model = Model(inputs=[si, fi], outputs=out)
    return model


In [12]:
#compile model
model = rnn_mlp_early()
adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.)
stop = EarlyStopping(patience=15, verbose=1)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy, f1])
checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, stop]

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 40)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256)          173056      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6117)         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 6373)         0           bidirectional_1[0][0]            
                                                                 input_2[0][0]                    
__________

In [13]:
#training
model.fit(x=[X_smiles_train, X_fingerprints_train], 
          y=y_train, 
          batch_size=32, 
          epochs=1000, 
          verbose=1, 
          callbacks=callbacks_list, 
          validation_data=([X_smiles_val, X_fingerprints_val], y_val), 
          shuffle=True)

Train on 2080 samples, validate on 446 samples
Epoch 1/1000
2080/2080 [==============================] - 395s 190ms/step - loss: 1.0055 - sparse_categorical_accuracy: 0.5567 - f1: 0.7724 - val_loss: 0.6939 - val_sparse_categorical_accuracy: 0.6592 - val_f1: 0.7905

Epoch 00001: val_loss improved from inf to 0.69391, saving model to rnn_mlp_early_tpk_src.h5
Epoch 2/1000
2080/2080 [==============================] - 381s 183ms/step - loss: 0.7729 - sparse_categorical_accuracy: 0.5774 - f1: 0.7720 - val_loss: 0.6428 - val_sparse_categorical_accuracy: 0.6592 - val_f1: 0.7905

Epoch 00002: val_loss improved from 0.69391 to 0.64278, saving model to rnn_mlp_early_tpk_src.h5
Epoch 3/1000
2080/2080 [==============================] - 356s 171ms/step - loss: 0.7045 - sparse_categorical_accuracy: 0.6082 - f1: 0.7726 - val_loss: 0.6514 - val_sparse_categorical_accuracy: 0.6435 - val_f1: 0.7905

Epoch 00003: val_loss did not improve from 0.64278
Epoch 4/1000
2080/2080 [==============================]

In [14]:
#testing
from keras.models import load_model
trained_model = load_model(model_file, custom_objects={'f1': f1})
print(trained_model.metrics_names)

['loss', 'sparse_categorical_accuracy', 'f1']


In [15]:
trained_model.evaluate(x=[X_smiles_train, X_fingerprints_train], y=y_train)

2080/2080 [==============================] - 28s 14ms/step


[0.6656845753009503, 0.6326923076923077, 0.7720215595685519]

In [16]:
trained_model.evaluate(x=[X_smiles_val, X_fingerprints_val], y=y_val)

446/446 [==============================] - 6s 14ms/step


[0.6427841860082651, 0.6591928245775368, 0.7904687021345301]

In [17]:
trained_model.evaluate(x=[X_smiles_test, X_fingerprints_test], y=y_test)

446/446 [==============================] - 6s 13ms/step


[0.714876350265982, 0.6255605399875898, 0.7683763963759213]

In [18]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
pred_train = trained_model.predict(x=[X_smiles_train, X_fingerprints_train])[:,1]
pred_val = trained_model.predict(x=[X_smiles_val, X_fingerprints_val])[:,1]
pred_test = trained_model.predict(x=[X_smiles_test, X_fingerprints_test])[:,1]
np.savez(model_file[:-2] + 'npz', pred_train=pred_train, pred_val=pred_val, pred_test=pred_test)